In [16]:
import pandas as pd

df_acled = pd.read_csv('./ACLED/cleaned_acled.csv')
df_emdat = pd.read_csv('./EM-DAT/cleaned_em_dat.csv')
df_hdi   = pd.read_csv('./HDI/cleaned_hdi.csv')

df_acled.shape, df_emdat.shape, df_hdi.shape


((1130048, 14), (16188, 14), (6948, 4))

In [4]:
df_acled['event_date'] = pd.to_datetime(df_acled['event_date'], errors='coerce')
df_acled['week_number'] = df_acled['event_date'].dt.isocalendar().week

In [5]:
df_acled.head()

,Unnamed: 0,iso,country,region,location,event_id_cnty,event_date,year,disorder_type,event_type,sub_event_type,latitude,longitude,fatalities,week_number
0,0,180,Democratic Republic of Congo,Middle Africa,Mikenge,DRC27766,2022-12-31,2022,Political violence,Violence against civilians,Abduction/forced disappearance,-3.4497,28.4476,0,52
1,1,710,South Africa,Southern Africa,Johannesburg,SAF18067,2022-12-31,2022,Political violence,Violence against civilians,Attack,-26.2023,28.0436,1,52
2,2,706,Somalia,Eastern Africa,Camaara,SOM38913,2022-12-31,2022,Political violence,Battles,Armed clash,5.1143,47.9303,7,52
3,3,728,South Sudan,Eastern Africa,Partet,SSD8950,2022-12-31,2022,Strategic developments,Strategic developments,Looting/property destruction,7.9236,32.0047,0,52
4,4,728,South Sudan,Eastern Africa,Mundari Bura,SSD8955,2022-12-31,2022,Political violence,Violence against civilians,Attack,5.6756,30.9748,3,52


In [8]:
df_acled['event_type'].unique()

array(['Violence against civilians', 'Battles', 'Strategic developments',
       'Protests', 'Riots', 'Explosions/Remote violence'], dtype=object)

In [ ]:
#weekly conflict_data dataset grouped by country, event_type, number of fatalities and number of events
conflict_data = df_acled.groupby(['year', 'week_number', 'country', 'event_type']).agg(
    num_deaths_conflict=('fatalities', 'sum'),   # Sum of 'value'
    num_conflicts=('event_id_cnty', 'count') # Count of events
).reset_index()

conflict_data.head()

,year,week_number,country,event_type,num_deaths_conflict,num_conflicts
0,2022,1,Afghanistan,Battles,51,22
1,2022,1,Afghanistan,Explosions/Remote violence,3,2
2,2022,1,Afghanistan,Protests,0,13
3,2022,1,Afghanistan,Strategic developments,0,5
4,2022,1,Afghanistan,Violence against civilians,11,22


In [9]:
conflict_data.shape

(64094, 6)

In [ ]:
# First, normalize the event_type values to clean column names
conflict_data['event_type_clean'] = conflict_data['event_type'].str.lower().str.replace(r'[^a-z0-9]+', '_', regex=True)

# Pivot deaths
deaths_pivot = conflict_data.pivot_table(
    index=['year', 'week_number', 'country'],
    columns='event_type_clean',
    values='num_deaths_conflict',
    aggfunc='sum'
).add_prefix('num_deaths_')

# Pivot conflicts
conflicts_pivot = conflict_data.pivot_table(
    index=['year', 'week_number', 'country'],
    columns='event_type_clean',
    values='num_conflicts',
    aggfunc='sum'
).add_prefix('num_conflicts_')

# Combine them
df_conflict = pd.concat([deaths_pivot, conflicts_pivot], axis=1).reset_index()

df_conflict.fillna(0, inplace=True)

df_conflict.head()

event_type_clean,year,week_number,country,num_deaths_battles,num_deaths_explosions_remote_violence,num_deaths_protests,num_deaths_riots,num_deaths_strategic_developments,num_deaths_violence_against_civilians,num_conflicts_battles,num_conflicts_explosions_remote_violence,num_conflicts_protests,num_conflicts_riots,num_conflicts_strategic_developments,num_conflicts_violence_against_civilians
0,2022,1,Afghanistan,51.0,3.0,0.0,0.0,0.0,11.0,22.0,2.0,13.0,0.0,5.0,22.0
1,2022,1,Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,2022,1,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0
3,2022,1,Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,2022,1,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,2.0,0.0,0.0


In [12]:
df_conflict.columns

Index(['year', 'week_number', 'country', 'num_deaths_battles',
       'num_deaths_explosions_remote_violence', 'num_deaths_protests',
       'num_deaths_riots', 'num_deaths_strategic_developments',
       'num_deaths_violence_against_civilians', 'num_conflicts_battles',
       'num_conflicts_explosions_remote_violence', 'num_conflicts_protests',
       'num_conflicts_riots', 'num_conflicts_strategic_developments',
       'num_conflicts_violence_against_civilians'],
      dtype='object', name='event_type_clean')

In [14]:
df_conflict.shape

(24878, 15)

In [22]:
df_emdat.dtypes

ISO               object
Country           object
Disaster Type     object
Total Deaths       int64
No. Injured        int64
No. Affected       int64
Start Year         int64
Start Month        Int64
Start Day          Int64
End Year           int64
End Month        float64
End Day          float64
Entry Date        object
Last Update       object
dtype: object

In [ ]:
# Combine into datetime
# Fill missing month with 1
df_emdat['Start Month'] = df_emdat['Start Month'].astype('Int64')
df_emdat['Start Day'] = df_emdat['Start Day'].astype('Int64')
df_emdat['Start Year'] = df_emdat['Start Year'].astype('Int64')
df_emdat['start_date'] = pd.to_datetime(
    df_emdat[['Start Year', 'Start Month', 'Start Day']].rename(
        columns={'Start Year': 'year', 'Start Month': 'month', 'Start Day': 'day'}
    ),
    errors='coerce'
)

# Extract week number
df_emdat['week_number'] = df_emdat['start_date'].dt.isocalendar().week

In [27]:
df_emdat.dtypes

ISO                      object
Country                  object
Disaster Type            object
Total Deaths              int64
No. Injured               int64
No. Affected              int64
Start Year                Int64
Start Month               Int64
Start Day                 Int64
End Year                  int64
End Month               float64
End Day                 float64
Entry Date               object
Last Update              object
start_date       datetime64[ns]
week_number              UInt32
dtype: object

In [28]:
#weekly disaster_data dataset grouped by country, disaster_type, number of fatalities and number of events
disaster_data = df_emdat.groupby(['week_number', 'Start Year', 'Country']).agg(
    num_disaster=('week_number', 'count'), # Count of events
    num_deaths_disaster=('Total Deaths', 'sum'),   # Sum of 'value'
    num_injured_disaster=("No. Injured", 'sum'),
    num_affected_disaster=("No. Affected", 'sum')
).reset_index()

disaster_data = disaster_data.rename(columns={"Start Year": "year", "Country": "country"})

disaster_data.head()

,week_number,year,country,num_disaster,num_deaths_disaster,num_injured_disaster,num_affected_disaster
0,1,2000,Angola,1,31,0,70000
1,1,2000,China,1,12,18,0
2,1,2000,India,1,10,0,0
3,1,2000,Indonesia,2,26,22,0
4,1,2000,Kenya,1,3,0,189


In [80]:
disaster_data.shape

(14217, 7)

In [31]:
disaster_data.columns

Index(['week_number', 'year', 'country', 'num_disaster', 'num_deaths_disaster',
       'num_injured_disaster', 'num_affected_disaster'],
      dtype='object')

In [40]:
disaster_data[disaster_data['year'] > 2021].shape

(1718, 7)

In [81]:
# Merge datasets
df_merged = pd.merge(
    df_conflict,
    disaster_data,
    on=['year', 'week_number', 'country'],
    how='outer'
)

df_merged.head()

,year,week_number,country,num_deaths_battles,num_deaths_explosions_remote_violence,num_deaths_protests,num_deaths_riots,num_deaths_strategic_developments,num_deaths_violence_against_civilians,num_conflicts_battles,num_conflicts_explosions_remote_violence,num_conflicts_protests,num_conflicts_riots,num_conflicts_strategic_developments,num_conflicts_violence_against_civilians,num_disaster,num_deaths_disaster,num_injured_disaster,num_affected_disaster
0,2000,1,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,31.0,0.0,70000.0
1,2000,1,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,12.0,18.0,0.0
2,2000,1,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10.0,0.0,0.0
3,2000,1,Indonesia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,26.0,22.0,0.0
4,2000,1,Kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.0,0.0,189.0


In [64]:
df_merged.shape

(37798, 19)

In [82]:
df_merged = df_merged[df_merged['year'] > 2021]

In [83]:
df_merged.shape

(25299, 19)

In [84]:
df_merged.head()

,year,week_number,country,num_deaths_battles,num_deaths_explosions_remote_violence,num_deaths_protests,num_deaths_riots,num_deaths_strategic_developments,num_deaths_violence_against_civilians,num_conflicts_battles,num_conflicts_explosions_remote_violence,num_conflicts_protests,num_conflicts_riots,num_conflicts_strategic_developments,num_conflicts_violence_against_civilians,num_disaster,num_deaths_disaster,num_injured_disaster,num_affected_disaster
12499,2022,1,Afghanistan,51.0,3.0,0.0,0.0,0.0,11.0,22.0,2.0,13.0,0.0,5.0,22.0,<NA>,NaN,NaN,NaN
12500,2022,1,Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,<NA>,NaN,NaN,NaN
12501,2022,1,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,<NA>,NaN,NaN,NaN
12502,2022,1,Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,<NA>,NaN,NaN,NaN
12503,2022,1,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,2.0,0.0,0.0,<NA>,NaN,NaN,NaN


In [38]:
df_merged.isna().sum()

year                                            0
week_number                                     0
country                                         0
num_deaths_battles                            421
num_deaths_explosions_remote_violence         421
num_deaths_protests                           421
num_deaths_riots                              421
num_deaths_strategic_developments             421
num_deaths_violence_against_civilians         421
num_conflicts_battles                         421
num_conflicts_explosions_remote_violence      421
num_conflicts_protests                        421
num_conflicts_riots                           421
num_conflicts_strategic_developments          421
num_conflicts_violence_against_civilians      421
num_disaster                                23581
num_deaths_disaster                         23581
num_injured_disaster                        23581
num_affected_disaster                       23581
dtype: int64

In [39]:
df_merged[~df_merged['num_disaster'].isna()].shape

(1718, 19)

In [42]:
df_merged.shape

(25299, 19)

In [41]:
df_hdi.head()

,iso,country,year,HDI
0,AFG,Afghanistan,1990,0.284
1,ALB,Albania,1990,0.649
2,DZA,Algeria,1990,0.593
3,AND,Andorra,1990,0.815
4,AGO,Angola,1990,0.369


In [85]:
#merge with hdi
df_merged = pd.merge(df_merged, df_hdi, on=['year', 'country'], how='left')

In [86]:
df_merged.shape

(25299, 21)

In [45]:
df_merged.head()

,year,week_number,country,num_deaths_battles,num_deaths_explosions_remote_violence,num_deaths_protests,num_deaths_riots,num_deaths_strategic_developments,num_deaths_violence_against_civilians,num_conflicts_battles,...,num_conflicts_protests,num_conflicts_riots,num_conflicts_strategic_developments,num_conflicts_violence_against_civilians,num_disaster,num_deaths_disaster,num_injured_disaster,num_affected_disaster,iso,HDI
0,2022,1,Afghanistan,51.0,3.0,0.0,0.0,0.0,11.0,22.0,...,13.0,0.0,5.0,22.0,<NA>,NaN,NaN,NaN,AFG,0.462
1,2022,1,Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,<NA>,NaN,NaN,NaN,ALB,0.789
2,2022,1,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,0.0,0.0,0.0,<NA>,NaN,NaN,NaN,DZA,0.745
3,2022,1,Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,<NA>,NaN,NaN,NaN,AGO,0.591
4,2022,1,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23.0,2.0,0.0,0.0,<NA>,NaN,NaN,NaN,ARG,0.849


In [48]:
df_merged.isna().sum()

year                                            0
week_number                                     0
country                                         0
num_deaths_battles                            421
num_deaths_explosions_remote_violence         421
num_deaths_protests                           421
num_deaths_riots                              421
num_deaths_strategic_developments             421
num_deaths_violence_against_civilians         421
num_conflicts_battles                         421
num_conflicts_explosions_remote_violence      421
num_conflicts_protests                        421
num_conflicts_riots                           421
num_conflicts_strategic_developments          421
num_conflicts_violence_against_civilians      421
num_disaster                                23581
num_deaths_disaster                         23581
num_injured_disaster                        23581
num_affected_disaster                       23581
iso                                          4086


In [87]:
df_merged.drop('iso', axis=1, inplace=True)

In [88]:
df_merged = df_merged.fillna(0)

In [89]:
df_merged.isna().sum()

year                                        0
week_number                                 0
country                                     0
num_deaths_battles                          0
num_deaths_explosions_remote_violence       0
num_deaths_protests                         0
num_deaths_riots                            0
num_deaths_strategic_developments           0
num_deaths_violence_against_civilians       0
num_conflicts_battles                       0
num_conflicts_explosions_remote_violence    0
num_conflicts_protests                      0
num_conflicts_riots                         0
num_conflicts_strategic_developments        0
num_conflicts_violence_against_civilians    0
num_disaster                                0
num_deaths_disaster                         0
num_injured_disaster                        0
num_affected_disaster                       0
HDI                                         0
dtype: int64

In [52]:
df_merged.columns

Index(['year', 'week_number', 'country', 'num_deaths_battles',
       'num_deaths_explosions_remote_violence', 'num_deaths_protests',
       'num_deaths_riots', 'num_deaths_strategic_developments',
       'num_deaths_violence_against_civilians', 'num_conflicts_battles',
       'num_conflicts_explosions_remote_violence', 'num_conflicts_protests',
       'num_conflicts_riots', 'num_conflicts_strategic_developments',
       'num_conflicts_violence_against_civilians', 'num_disaster',
       'num_deaths_disaster', 'num_injured_disaster', 'num_affected_disaster',
       'HDI'],
      dtype='object')

In [90]:
# Calculate the most_needs Index

# Conflict + conflict death columns
conflict_cols = [
    'num_deaths_battles',
    'num_deaths_explosions_remote_violence',
    'num_deaths_protests',
    'num_deaths_riots',
    'num_deaths_strategic_developments',
    'num_deaths_violence_against_civilians',
    'num_conflicts_battles',
    'num_conflicts_explosions_remote_violence',
    'num_conflicts_protests',
    'num_conflicts_riots',
    'num_conflicts_strategic_developments',
    'num_conflicts_violence_against_civilians'
]

# Disaster-related columns
disaster_cols = [
    'num_disaster',
    'num_deaths_disaster',
    'num_injured_disaster',
    'num_affected_disaster'
]

# Define a weighted formula for most_needs
df_merged['most_needs'] = (
    0.5 * df_merged[conflict_cols].sum(axis=1) +
    0.3 * df_merged[disaster_cols].sum(axis=1) +
    0.2 * (1 - df_merged['HDI'])  # Include inverse of HDI
)


In [91]:
df_merged.dtypes

year                                          Int64
week_number                                  UInt32
country                                      object
num_deaths_battles                          float64
num_deaths_explosions_remote_violence       float64
num_deaths_protests                         float64
num_deaths_riots                            float64
num_deaths_strategic_developments           float64
num_deaths_violence_against_civilians       float64
num_conflicts_battles                       float64
num_conflicts_explosions_remote_violence    float64
num_conflicts_protests                      float64
num_conflicts_riots                         float64
num_conflicts_strategic_developments        float64
num_conflicts_violence_against_civilians    float64
num_disaster                                  Int64
num_deaths_disaster                         float64
num_injured_disaster                        float64
num_affected_disaster                       float64
HDI         

In [92]:
df_merged.head()

,year,week_number,country,num_deaths_battles,num_deaths_explosions_remote_violence,num_deaths_protests,num_deaths_riots,num_deaths_strategic_developments,num_deaths_violence_against_civilians,num_conflicts_battles,...,num_conflicts_protests,num_conflicts_riots,num_conflicts_strategic_developments,num_conflicts_violence_against_civilians,num_disaster,num_deaths_disaster,num_injured_disaster,num_affected_disaster,HDI,most_needs
0,2022,1,Afghanistan,51.0,3.0,0.0,0.0,0.0,11.0,22.0,...,13.0,0.0,5.0,22.0,0,0.0,0.0,0.0,0.462,64.6076
1,2022,1,Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0,0.0,0.0,0.0,0.789,1.0422
2,2022,1,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.745,7.551
3,2022,1,Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0,0.0,0.0,0.0,0.591,1.0818
4,2022,1,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23.0,2.0,0.0,0.0,0,0.0,0.0,0.0,0.849,12.5302


In [ ]:
#from sklearn.preprocessing import MinMaxScaler

#non_normalized = ['country', 'year', 'week_number']
#cols_to_scale = [col for col in df_merged.columns if col not in non_normalized]

#scaler = MinMaxScaler()
#df_merged[cols_to_scale] = scaler.fit_transform(df_merged[cols_to_scale])


In [76]:
df_merged.head()

,year,week_number,country,num_deaths_battles,num_deaths_explosions_remote_violence,num_deaths_protests,num_deaths_riots,num_deaths_strategic_developments,num_deaths_violence_against_civilians,num_conflicts_battles,...,num_conflicts_protests,num_conflicts_riots,num_conflicts_strategic_developments,num_conflicts_violence_against_civilians,num_disaster,num_deaths_disaster,num_injured_disaster,num_affected_disaster,HDI,most_needs
0,2022,1,Afghanistan,0.018668,0.000949,0.0,0.0,0.0,0.012332,0.045643,...,0.010450,0.000000,0.025381,0.134969,0.0,0.0,0.0,0.0,0.477766,6.491927e-06
1,2022,1,Albania,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.003906,0.000000,0.000000,0.0,0.0,0.0,0.0,0.815926,7.410760e-08
2,2022,1,Algeria,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.012058,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.770424,7.312623e-07
3,2022,1,Angola,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000804,0.001953,0.000000,0.000000,0.0,0.0,0.0,0.0,0.611169,7.810578e-08
4,2022,1,Argentina,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.018489,0.003906,0.000000,0.000000,0.0,0.0,0.0,0.0,0.877973,1.233982e-06


In [77]:
df_merged.columns

Index(['year', 'week_number', 'country', 'num_deaths_battles',
       'num_deaths_explosions_remote_violence', 'num_deaths_protests',
       'num_deaths_riots', 'num_deaths_strategic_developments',
       'num_deaths_violence_against_civilians', 'num_conflicts_battles',
       'num_conflicts_explosions_remote_violence', 'num_conflicts_protests',
       'num_conflicts_riots', 'num_conflicts_strategic_developments',
       'num_conflicts_violence_against_civilians', 'num_disaster',
       'num_deaths_disaster', 'num_injured_disaster', 'num_affected_disaster',
       'HDI', 'most_needs'],
      dtype='object')

In [93]:
df_merged.to_csv('ccar_merged.csv')